In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 上的 SQL 程式碼產生

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/sql-code-generation/sql_code_generation.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory 標誌"><br> 在 Colab 上執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/sql-code-generation/sql_code_generation.zh.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub 標誌"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/sql-code-generation/sql_code_generation.zh.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI 標誌"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
| 作者 | [Roy Arsan](https://www.linkedin.com/in/arsan) |


## 概述
大語言模型可用於產生程式碼，包含 SQL。特定來說，模型可以將自然語言文字轉換成 SQL 查詢。常見的用途之一是，讓使用者在不需了解資料表名稱、資料架構或基礎資料倉庫 (例如 BigQuery) 的特定 SQL 方言或查詢引擎情況下查詢資料。

此筆記本涵蓋 SQL 程式碼產生提示工程的最佳實務做法，並實際演練來自 [SQL-PaLM：改良大型語言模型適應以轉換文字為 SQL](https://arxiv.org/pdf/2306.00739.pdf) 的技術。例如，會動態地擷取 BigQuery 資料集架構，並提供給提示當作背景，以了解 LLM 並客製化其輸出。筆記本也會示範簡單的模型評估，透過針對 BigQuery 資料集執行已產生的 SQL 查詢，並將它們與 ground 真實查詢及對應的結果進行比較，來評估。

對於此筆記本，你會產生 SQL 查詢來分析 Cloud Audit Logs，並回答有關你自己的 Google Cloud 計畫中活動的重要安全性問題。儘管此筆記本使用 BigQuery 日誌資料集，但這裡提出的概念和方法可以套用在其他資料庫和資料集。

![NL2SQL flow](https://services.google.com/fh/files/misc/nl2sql_for_log_analytics2.png)


### 目標

在筆記本結束時，你應能夠：

* 使用模型根據自然語言問題來產生 SQL 查詢：
  * 使用少次嘗試提示
  * 將自訂資料集架構作為脈絡提供
  * 將模型輸出格式化

* 評估模型所產生的查詢，方法是：
  * 對應實際資料集執行已清除的查詢
  * 使用簡單的模糊字串比對將查詢 (及其結果) 與基本真實查詢比較
  * 計算模型準確度分數

此外，你可以使用此筆記本來從你的稽核記錄中回答你自己的安全性問題，例如：

- 上個月任何使用者身分是否有異常高的雲端 API 使用量？
- 上週任何未經核准的身分是否有破壞性動作？
- 本週任何使用者存取的資料量是否有異常的每日激增現象？


## 開始使用


### 必備條件
如果你尚未執行此操作，唯一的需求是將你現有的記錄區塊 [升級](https://cloud.google.com/logging/docs/buckets#upgrade-bucket) 至 Log Analytics，這樣便能為你連結一個 BigQuery 資料集，裡面有可進行查詢的記錄資料。這是 **一鍵完成步驟，不會產生額外的費用** 。預設情況下，每個專案的 _Required_ 區塊會啟用、擷取並儲存 Cloud Audit 管理員活動記錄，不會產生任何費用。

![one click prerequisite](https://services.google.com/fh/files/misc/upgrade_log_bucket.png)


### 安裝 SDK


In [ ]:
# Install Vertex AI SDK to use for model predictions
!pip install google-cloud-aiplatform google-cloud-bigquery --upgrade --user

# Install fuzzy string comparison modules for model output evaluation
!pip install -q python-Levenshtein --upgrade --user
!pip install -q fuzzywuzzy --upgrade --user

僅當你想稍後將模型生成的 SQL 查詢及其結果儲存到 Google 試算表以供後續疑難排解或擴充範例資料集時，才安裝 Google 試算表的 Python SDK。這**不適用於 Vertex AI Workbench 或 Colab Enterprise** ，因為這些環境中的筆記本基於安全性考量，無法存取 Google 雲端硬碟或 Google 試算表。


In [ ]:
# [Optional] Install Python SDK for Google Sheets
!pip install gspread --upgrade --user

**僅 Colab：** 取消下一個Cell註解以重新啟動Kernel或使用按鈕重新啟動Kernel。對於 Vertex AI Workbench，你可以使用頂端的按鈕重新啟動終端機。


In [ ]:
# # Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 驗證你的筆記本電腦環境
* 如果你使用 **Colab** 執行此筆記本電腦，執行下列Cell並繼續。
* 如果你使用 **Vertex AI 工作台** ，請查看 [這裡](https://github.com/doggy8088/generative-ai/tree/main/setup-env) 的設定說明。


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
# For debug only
!gcloud config list --format 'value(core.account)'

### 匯入函式庫


**僅限 Colab：** 取消下方單元格的註解，以初始化 Vertex AI SDK。對於 Vertex AI Workbench，不需要執行此動作。


In [ ]:
# PROJECT_ID = "[your-project-id]" # @param {type:"string"}
# LOCATION = "us-central1" # @param {type:"string"}

# from google.cloud import aiplatform
# aiplatform.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
import numpy as np
import pandas as pd
from vertexai.language_models import TextGenerationModel

### 為 BigQuery 設定專案和資料集


這是包含下列項目的專案：
- 連結的 BigQuery 資料集 `BQ_LINKED_DATASET` (含你的原始記錄)，以及，
- 新的 BigQuery 資料集 `BQ_PROCESSED_DATASET`，用來儲存已處理的記錄。

這個專案可以是你用於 Vertex AI 的專案，也可以是不同的專案。

請確定你有 `BQ_LINKED_DATASET` 資料集的 **BigQuery Data Viewer** 角色


In [ ]:
BQ_PROJECT_ID = "[bq-project-id]"  # @param {type:"string"}
BQ_LINKED_DATASET = "[linked-bq-dataset]"  # @param {type:"string"}
BQ_PROCESSED_DATASET = "[new-bq-dataset]"  # @param {type:"string"}

from google.cloud import bigquery

client = bigquery.Client(project=BQ_PROJECT_ID)

### 載入模型


我們將會與 Vertex AI LLM model `text-bison@001`互動:


In [ ]:
MODEL_ID = "text-bison@001"  # @param {type:"string"}

model = TextGenerationModel.from_pretrained(MODEL_ID)

## 準備資料


> 如果你已使用 [Dataform 作為 Community Security Analytics 的一部分](https://github.com/GoogleCloudPlatform/security-analytics/tree/main/dataform) (CSA) 來處理原始記錄並使用經過整理的表格進行標準化，可以略過此部分。如需瞭解 CSA 以及如何自動持續建立經過後製處理的資料表以取代原始記錄，請查看 [Google Cloud 部落格文章](https://cloud.google.com/blog/products/data-analytics/deploy-community-security-analytics-with-dataform)。


如同任何其他的人工智慧 /機器學習專案，第一件事便是準備好資料，包括給少次提示和後續評估所用的資料集。你會將儲存在 BigQuery 連結資料集的原始記錄預先處理成摘要表格，並放到你的新 BigQuery 資料集裡。這個表格將以彙總方式顯示記錄，並正規化成一個簡單的架構。這讓你可以解鎖和縮放機器學習分析：
- 從運算觀點而言，因為這個資料集較小且簡單。
- 從人才觀點而言，因為研究人員和分析師不一定要熟悉原始記錄的複雜架構 ( [Log Entry 定義](https://cloud.google.com/logging/docs/reference/v2/rest/v2/LogEntry))。


### 建立新資料集


In [ ]:
!bq --location=US mk --dataset {BQ_PROJECT_ID}:{BQ_PROCESSED_DATASET}

### 建立使用者行為表格


讓我們搜尋並處理稽核記錄以建立使用者動作的表格，按天彙總。這個摘要表格會將集合縮減至相關的記錄並簡化結構，而這也會簡化探索性和進階分析。對於有興趣的人士，我們使用與 CSA 存放區中所定義相同的 SQL 查詢，特別是 [`csa_4_01_summary_daily`](https://github.com/GoogleCloudPlatform/security-analytics/blob/main/dataform/definitions/summary/csa_4_01_summary_daily.sqlx) Dataform 定義檔案。


In [ ]:
TABLE_NAME = "csa_4_01_summary_daily"
TABLE_ID = f"{BQ_PROJECT_ID}.{BQ_PROCESSED_DATASET}.{TABLE_NAME}"
SUMMARY_LOOKBACK_DAYS = 90

job_config = bigquery.QueryJobConfig(
    destination=TABLE_ID, write_disposition="WRITE_TRUNCATE"
)

sql = f"""
SELECT
  EXTRACT(DATE FROM timestamp) AS day,
  proto_payload.audit_log.authentication_info.principal_email,
  ARRAY_AGG(DISTINCT proto_payload.audit_log.method_name IGNORE NULLS) AS actions,
  COUNT(*) AS counter
FROM `{BQ_PROJECT_ID}.{BQ_LINKED_DATASET}._AllLogs`
WHERE
  timestamp >=  TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL {SUMMARY_LOOKBACK_DAYS} DAY)
  AND proto_payload.audit_log.authentication_info.principal_email IS NOT NULL
  AND proto_payload.audit_log.method_name NOT LIKE "storage.%.get"
  AND proto_payload.audit_log.method_name NOT LIKE "v1.compute.%.list"
  AND proto_payload.audit_log.method_name NOT LIKE "beta.compute.%.list"
GROUP BY
  day,
  proto_payload.audit_log.authentication_info.principal_email
"""

# Start the query and save results in new table
query_job = client.query(sql, job_config=job_config)
result = query_job.result()  # Wait for the job to complete.

print(f"{result.total_rows} user action records loaded to table {TABLE_ID}")

### 匯入樣本查詢


你現在會從CSV文件裡擷取15個範例安全問題和相應的SQL查詢清單。這些安全問題是變動自開原始碼[社群安全分析](https://github.com/GoogleCloudPlatform/security-analytics)的。CSA提供一組安全問題以及對應的查詢，適用於BigQuery、Log Analytics和Chronicle。

我們會使用這些查詢的子集作為模型提示範例中的小樣本，以及將剩餘的部分用於模型評估。

執行下列命令，從GCS儲存區讀取CSV文件，並載入所有記錄至記憶中的pandas DataFrame中：


In [ ]:
BUCKET_ID = "csa-datasets-public"  # @param {type:"string"}
FILENAME = "SQL_Generator_Example_Queries.csv"  # @param {type:"string"}

df = pd.read_csv(f"gs://{BUCKET_ID}/{FILENAME}", header=0)
df.head(2)

檢索範例問題所引用的表格名稱。這應該與我們在上方建立的已後製處理日誌表格相同。


In [ ]:
# Retrieve unique table names excluding null values and empty string
BQ_TABLES = df["Qualified table name"].replace("", np.nan).dropna().unique()
print(BQ_TABLES)

### 萃取訓練 & 評估資料集


提取訓練和評估資料集，並儲存在各自的資料架構中：


In [ ]:
train_df = df.loc[df["Dataset"] == "Train", ["Question", "SQL Query"]]
eval_df = df.loc[df["Dataset"] == "Eval", ["Question", "SQL Query"]]

來看看個別分類中的幾則紀錄:


In [ ]:
train_df.head(5)

In [ ]:
eval_df.head(5)

## 準備幾次提示


### 定義提示範本


模型提示將包含以下組成部分：
1. 簡潔的敘述以指定任務
1. 架構定義以描述現有資料集
1. 幾個自然語言問句和對應的 SQL 陳述範例


這是我們以後會使用的樣板，使用這 3 個組成部分產生提示。


In [ ]:
# This string template takes three arguments:
# - schema definition
# - few shot examples
# - question for which query needs to be generated

prompt_template = """\
This is a task converting text into GoogleSQL statement.
We will first give you the dataset schema and then ask a question in text.
You are asked to generate SQL statement which is valid for BigQuery.
Remove any delimiters around answer such as "```"

BigQuery tables schema definition:
{schema_definition}
Here are a few shot examples:
{few_examples}
Write GoogleSQL query for following question: {question}
Answer: "Query here"
"""

### 建構 schema 定義 (精簡版）


首先，我們需要建立簡潔的資料集架構定義。如同前面所述，我們會將它用作提示背景的一部分來解決結果。


從 BigQuery 資料集的 `INFORMATION_SCHEMA` 中擷取資料表和欄位定義。


In [ ]:
# Following SQL query will generate schema definition of your dataset
QUERY = f"""\
SELECT
    '[Schema (values)]: ' || '| log_summary | ' || STRING_AGG(table_values, ' | ') || ';' AS tables_definition,
    '[Column names (type)]: ' || STRING_AGG(column_names_types) || ';' AS columns_definition
FROM (
    SELECT
      table_name,
      table_name || ' : ' || STRING_AGG(column_name, ' , ') as table_values,
      STRING_AGG(table_name || ' : ' || column_name || ' (' || data_type || ')', ' | ') as column_names_types
    FROM {BQ_PROJECT_ID}.{BQ_PROCESSED_DATASET}.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
    WHERE table_name IN {'(' + ",".join(map(lambda x: f"'{x}'", BQ_TABLES)) + ')'}
    GROUP BY table_name
    ORDER BY table_name
)
"""

# Create query job
query_job = client.query(QUERY)
# Get first row
schema = next(query_job.result())

# Build schema definition
schema_definition = f"""\
{schema.tables_definition}

{schema.columns_definition}
"""

print(schema_definition)

### 將查詢新增為少量範例


這是一個用於格式化範例輸入和輸出在提示中的輔助函式。 我們會在將圖片逐步加入提示中時使用此輔助函式。


In [ ]:
one_shot_template = """
Question: {question}

Answer: {query}
"""

從我們資料集新增查詢，作為提示的範例：


In [ ]:
few_examples = ""
for index, row in train_df.iterrows():
    few_examples += one_shot_template.format(
        question=row["Question"], query=row["SQL Query"]
    )

print(f"Added {str(train_df.shape[0])} pairs as few-shot examples")

### 檢視完整提示


使用模式定義、少樣本範例和要回答的範例問題，我們以產生完整提示為例


In [ ]:
question = "This is a sample question"

prompt = prompt_template.format(
    schema_definition=schema_definition, few_examples=few_examples, question=question
)
print("Prompt:")
print(prompt)

print("Number of input tokens: " + str(len(prompt)))

## 生成 SQL 查詢


### 定義用於產生 SQL 的輔助函式


下列輔助函式 `generate_sql()` 用於使用我們迄今建立的提示範本從 Vertex AI LLM 模型擷取 SQL 查詢。

請注意 `generate_sql()` 如何使用 `sanitize_output()` 函式將回應整理成 SQL 查詢本身，然後再傳回結果。即使模型提示包含調整模型輸出的說明，仍然可能有需要去除的封閉式引號或程式區塊反引號，以避免後續的 SQL 語法錯誤。


In [ ]:
import re


# Strip text to include only the SQL code block with
def sanitize_output(text: str) -> str:
    # Strip whitespace and any potential backticks enclosing the code block
    text = text.strip()
    regex = re.compile(r"^\s*```(\w+)?|```\s*$")
    text = regex.sub("", text).strip()

    # Find and remove any trailing quote without corresponding opening quote
    if re.search(r'^[^"]*"$', text):
        text = text[:-1]
    # Find and remove any leading quote without corresponding closing quote
    if re.search(r'^"[^"]*$', text):
        text = text[1:]

    return text


# Call model using prompt and pre-defined parameters
def generate_sql(
    model,
    prompt: str,
    temperature: float = 0.2,
    max_output_tokens: int = 1024,
    top_k: int = 40,
    top_p: float = 0.8,
) -> str:
    print("Generating SQL...")
    print("Number of input tokens: " + str(len(prompt)))

    response = model.predict(
        prompt,
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        top_k=top_k,
        top_p=top_p,
    )

    text = response.text
    print("Number of output tokens: " + str(len(text)))
    print("Response:")
    print(text)

    # Strip text to include only the SQL code block
    text = sanitize_output(text)
    print("Response stripped:")
    print(text)

    return text

### 定義執行 SQL 的輔助函式


以下輔助函式 `execute_sql()` 用於對 BigQuery 數據集執行 SQL 查詢，並以資料框回傳結果。

請注意 `execute_sql()` 如何確保使用上面指定的專案和 BigQuery 數據集來限定資料表名稱，然後才執行 SQL 查詢。


In [ ]:
# Limit number of bytes processed as a guardrail for cost control
BQ_MAX_BYTES_BILLED = pow(2, 30)  # 1GB


def execute_sql(query: str):
    print("Executing SQL...")

    # Qualify table names with your project and dataset ID
    for table_name in BQ_TABLES:
        query = query.replace(
            table_name, f"{BQ_PROJECT_ID}.{BQ_PROCESSED_DATASET}.{table_name}"
        )

    print("Query:")
    print(query)

    # Validate the query by performing a dry run without incurring a charge
    job_config = bigquery.QueryJobConfig(use_query_cache=False, dry_run=True)
    try:
        response = client.query(query, job_config=job_config)
    except Exception as e:
        print("Error validating query:")
        print(e)
        return e

    print("Query will process {:.2f} KB.".format(response.total_bytes_processed / 1024))

    # Execute the query
    job_config = bigquery.QueryJobConfig(
        use_query_cache=False, maximum_bytes_billed=BQ_MAX_BYTES_BILLED
    )
    try:
        response = client.query(query)
        df = response.to_dataframe()
    except Exception as e:
        print("Error executing query:")
        print(e)
        return e

    return df

### 範例 1


讓我們生成 SQL 來回答這個範例問題：

*列出包含「刪除」或「移除」字詞的所有使用者操作，時段涵蓋上個月。在結果中包含使用者與日期。*


In [ ]:
question = "List all user actions that contains the word 'delete' or 'remove' over the last month. Include the user and the day in the results."

query = generate_sql(
    model,
    prompt_template.format(
        schema_definition=schema_definition,
        few_examples=few_examples,
        question=question,
    ),
)

現在，我們用 BigQuery 中的即時資料集，來驗證已產生的查詢。


In [ ]:
# Execute the query
query_result = execute_sql(query)
display(query_result)

### 範例 2


我們將產生 SQL 來回答此範例問題：

*列出在過去 7 天內任何未核可使用者進行的，包含不區分大小寫的 IAM 動作，其中核可使用者包含 'admin@example.com'。*


In [ ]:
question = "List any action containing IAM case-insensitive by any unapproved user over the last 7 days, where approved user include 'admin@example.com'"

query = generate_sql(
    model,
    prompt_template.format(
        schema_definition=schema_definition,
        few_examples=few_examples,
        question=question,
    ),
)

讓我們針對你的 BigQuery 資料集測試產生的查詢：


In [ ]:
# Execute the query
query_result = execute_sql(query)
display(query_result)

## 評估模型


### 在評估資料集上運行模型


讓我們為評估資料集中所有的問題產生 SQL 查詢。該資料集同時包括「問題」和正解「SQL 查詢」。執行以下程式碼，為資料集中的每個問題自動呼叫模型，並將回應記錄在新的欄位「已產生的 SQL 查詢」。由於模型呼叫是串列執行的，這可能需要幾分鐘。


In [ ]:
eval_df["Generated SQL Query"] = eval_df["Question"].apply(
    lambda x: generate_sql(
        model,
        prompt_template.format(
            schema_definition=schema_definition, few_examples=few_examples, question=x
        ),
    )
)

eval_df

In [ ]:
len(eval_df)

### 執行真實查詢


咱們在評估生成查詢之前，在你的資料集內執行「準確度」查詢。


In [ ]:
eval_df["SQL Query Result"] = eval_df["SQL Query"].apply(execute_sql)

讓我們來看看結果


In [ ]:
eval_df.loc[:, ["SQL Query", "SQL Query Result"]]

### 執行產生的查詢


In [ ]:
eval_df["Generated SQL Query Result"] = eval_df["Generated SQL Query"].apply(
    execute_sql
)

讓大家瞧瞧部分結果


In [ ]:
eval_df.loc[:, ["Generated SQL Query", "Generated SQL Query Result"]].head()

### 結果序號化


由於每次成功查詢的結果在評估 DataFrame 中都表示為嵌套式 DataFrame，因此需要先序列化它們，這樣才能比較容易地比較結果，並選擇儲存它們在試算表或 CSV 檔案中。


In [ ]:
def format_query_result(query_result):
    if isinstance(query_result, pd.DataFrame):
        if query_result.shape[0] == 0:
            return "No results found"
        return query_result.to_csv(index=False)
    elif isinstance(query_result, Exception):
        return query_result.message
    else:
        return query_result


eval_df["Generated SQL Query Result Formatted"] = eval_df[
    "Generated SQL Query Result"
].apply(format_query_result)
eval_df["SQL Query Result Formatted"] = eval_df["SQL Query Result"].apply(
    format_query_result
)

檢查結果


In [ ]:
eval_df.head()

### 計算匹配分數


針對每一組查詢，計算匹配比分，以評估模型準確度。以我們的情況來看，我們將根據每個產生式查詢與相對應的 ground truth 查詢模糊字串配對，來計算分數。


讓我們匯入並使用已與 Levenshtein 模組一起安裝的 `fuzzywuzzy` 函式庫，以快速計算兩個字串之間的 Levenshtein 距離：


In [ ]:
from fuzzywuzzy import fuzz


def get_match_of_queries(df):
    return fuzz.partial_ratio(df["SQL Query"], df["Generated SQL Query"])


eval_df["match_score_queries"] = eval_df.apply(get_match_of_queries, axis=1)
eval_df.loc[:, ["SQL Query", "Generated SQL Query", "match_score_queries"]]

現在計算所有問題的平均分數


In [ ]:
print(
    "The average match score based on raw generated queries is: ",
    round(eval_df["match_score_queries"].mean(), 2),
    "%",
)

產生的查詢可能會不同於『真實』查詢的 SQL 實作或變數命名，但仍能提供安全性問題的正確答案。因此，計算每個查詢字串與其對應『真實』查詢字串的比對分數並非足夠的評估方式。因此，我們也會計算從你的資料集回傳的實際結果之間的比對分數。


讓我們對結果的已格式化版本執行模糊匹配邏輯。已格式化的欄已字串化並準備好進行字串比較。根據比較實際結果的大小，可能需要幾分鐘才能完成。


In [ ]:
def get_match_of_results(df):
    return fuzz.partial_ratio(
        df["SQL Query Result Formatted"], df["Generated SQL Query Result Formatted"]
    )


# This may take several minutes to complete
eval_df["match_score"] = eval_df.apply(get_match_of_results, axis=1)
eval_df.loc[
    :,
    [
        "SQL Query",
        "Generated SQL Query",
        "SQL Query Result Formatted",
        "Generated SQL Query Result Formatted",
        "match_score_queries",
        "match_score",
    ],
]

現在計算所有問題的平均分數


In [ ]:
print(
    "The average match score of all generated queries is: ",
    round(eval_df["match_score"].mean(), 2),
    "%",
)

## [可選] 將結果儲存至 Google 試算表


你可以將所有已產生的查詢、結果和分數儲存到 Google 試算表中，以利視覺檢查和將來參考，例如追蹤模型準確度。

如果你使用 **Vertex AI Workbench** 或 **Colab Enterprise** ，可以略過這個區段，因為這些環境中的筆記本基於安全性考量無法存取 Google 雲端硬碟或 Google 試算表。


在 Google 試算表中建立一個新的試算表 (https://sheets.new)，並將你的唯一試算表 ID 複製至 `QUERIES_SHEET_ID` 參數中。你可以在 Google 試算表網址中找到試算表 ID：docs.google.com/spreadsheets/d/*spreadsheetId*/edit#gid=0


In [ ]:
QUERIES_SHEET_ID = ""  # @param {type:"string"}
QUERIES_WORKSHEET_NAME = "Evaluation Dataset"  # @param {type:"string"}

In [ ]:
import gspread
from google.auth import default

# Authenticate with Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)

wks_results = gc.open_by_key(QUERIES_SHEET_ID).worksheet(QUERIES_WORKSHEET_NAME)

# Drop Query Result column which may contain non-serializable objects
eval_df_copy = eval_df.drop(columns=["SQL Query Result", "Generated SQL Query Result"])


def limit_cell_length(cell) -> str:
    if len(cell) >= 50000:
        return cell[:49990] + "..."
    return cell


eval_df_copy["Generated SQL Query Result Formatted"] = eval_df_copy[
    "Generated SQL Query Result Formatted"
].apply(limit_cell_length)
eval_df_copy["SQL Query Result Formatted"] = eval_df_copy[
    "SQL Query Result Formatted"
].apply(limit_cell_length)

wks_results.update(
    [eval_df_copy.columns.values.tolist()] + eval_df_copy.values.tolist()
)

## 摘要


在這個筆記本中，你已能夠：
- 準備資料集，包括將日誌寫入摘要表，以進行更輕鬆、更快速的日誌分析。
- 使用現有文字：SQL 對集作為少量範例，結合動態檢索的內容 (例如，資料庫架構)，建立提示模板，以基礎模型。
- 使用語言模型和少數提示範例，將 NL 問題轉換成 SQL 查詢。
- 清理和驗證模型輸出。
- 透過在 BigQuery 上執行已產生的查詢，評估模型輸出。
- 在整個評估資料集上執行模型。
- 根據模型輸出計算比對分數，也就是將產生的查詢與真實查詢做比較。
- 根據產生的查詢的實際結果計算比對分數，與真實查詢的結果做比較。
- 儲存結果，以供未來參考或追蹤模型準確度。


## 清理


如要清除此筆記本中使用的所有 Google Cloud 資源，你可以刪除你用於此教學課程的 [Google Cloud 專案](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否則，你可以刪除在此教學課程中建立的個別資源，即包含已處理資料的大查詢資料集 `BQ_PROCESSED_DATASET`：


In [ ]:
# Delete the created BigQuery dataset
!bq rm -r -f {BQ_PROJECT_ID}:{BQ_PROCESSED_DATASET}